In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from rich.progress import Progress

from losses import BinaryCrossentropy
from cifar_10_dataset_loading import load_cifar_10
from layers import Convolutional, Linear, Relu, Sigmoid, Flatten

In [2]:
x_train, y_train, x_test, y_test = load_cifar_10()

In [3]:
classes = y_train.argmax(axis=1)
planes_idx = np.arange(classes.shape[0])[classes == 0]

In [4]:
IMGS_IDX = [0, 351, 5673, 5494]
x = x_train[IMGS_IDX]
y = y_train[IMGS_IDX]
px.imshow(x, facet_col=0)

In [13]:
nn: list[Convolutional|Relu|Linear|Sigmoid] = [
    Convolutional((10, 5, 5, 3)),
    Relu(),
    Convolutional((10, 3, 3, 10)),
    Relu(),
    Flatten(),
    Linear(6760, 64),
    Relu(),
    Linear(64, y.shape[1]),
]
loss = BinaryCrossentropy()

In [6]:
def forward(activation:np.ndarray) -> np.ndarray:
    for layer in nn:
        activation = layer.forward(activation)
    return activation

def backward(gradient:np.ndarray, learning_rate:float) -> np.ndarray:
    for i, layer in enumerate(reversed(nn)):
        gradient = layer.backward(gradient, learning_rate)
        # print(gradient.shape)

def compute_accuracy(activation:np.ndarray, labels:np.ndarray) -> float:
    return (activation.argmax(axis=1) == labels.argmax(axis=1)).mean()

In [7]:
activations = forward(x)
y_true = activations.argmax(axis=1) == y.argmax(axis=1)

y_true

array([False, False,  True,  True])

In [8]:
stats = {
    "losses": [],
    "accuracies": [],
}

In [14]:
NB_EPOCHS = 30
LEARNING_RATE = 0.02
BATCH_SIZE = 128

# with Progress() as progress:
#     task = progress.add_task("Training", total=NB_EPOCHS * x.shape[0] // BATCH_SIZE)
for epoch in range(NB_EPOCHS):
    for i in range(0, x.shape[0], BATCH_SIZE):
        batch = x[i:i+BATCH_SIZE]
        labels = y[i:i+BATCH_SIZE]
        activations = forward(batch)
        print(activations)
        mean_loss = loss.forward(activations, labels)
        gradients = loss.backward(activations, labels)
        backward(gradients, LEARNING_RATE)
        # progress.update(task, advance=1, description=f"loss: {mean_loss:2f}")
    dataset_activations = forward(x)
    mean_dataset_loss = loss.forward(dataset_activations, y)
    stats["losses"].append(mean_dataset_loss)
    stats["accuracies"].append(compute_accuracy(dataset_activations, y))

[[-7.02557434e-05 -5.89821684e-05  2.97413677e-05 -2.60416239e-05
  -1.19856459e-04  1.00156311e-04 -3.71689567e-05  4.52902543e-05
  -2.26862828e-05 -2.63437135e-04]
 [-6.24965085e-05 -1.68488657e-05  1.01335905e-05 -2.58395879e-05
  -9.04807589e-05  1.06462676e-04 -2.02002702e-06  2.31295931e-05
  -1.26637032e-05 -1.83823347e-04]
 [-8.80063063e-05 -1.90150507e-05  7.71811218e-06  2.28955081e-05
  -9.67161806e-05  3.23700155e-05  6.38267032e-05 -4.10167327e-05
   9.63713220e-06 -2.41722744e-04]
 [-6.60292517e-05 -3.20645575e-05  5.90871228e-05 -5.93445283e-05
  -5.41088497e-05  8.77520819e-05 -9.51454915e-06  5.08249433e-05
  -1.60359228e-05 -1.65078482e-04]]
[[ 1.59648651e+89 -4.70271973e+82 -2.37507072e+82 -4.43503367e+82
  -7.29635437e+82 -1.61830914e+80  1.88532869e+89 -2.73126312e+82
  -3.87418059e+82 -1.25057544e+83]
 [ 1.08177335e+89 -3.18648754e+82 -1.61011459e+82 -3.00477817e+82
  -4.94429986e+82 -1.17793932e+80  1.27735546e+89 -1.85089577e+82
  -2.62288168e+82 -8.47599501e+8

In [ ]:
fig = (
    px.scatter(
        (
            pd.DataFrame(stats)
            .assign(batch=range(len(stats["losses"])))
            .melt(id_vars="batch", value_vars=["losses", "accuracies"])
        ),
        x="batch",
        y="value",
        color="variable",
        facet_row="variable",
        title="Training Metrics Over Btaches",
        height=600,
    )
    .update_yaxes(matches=None)
    .update_yaxes(showticklabels=True)
)
fig.show()